In [ ]:
# Standard imports
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import tqdm

# Hugging Face Hub import

# Diffusers-specific imports
from diffusers import StableDiffusionPipeline, DDIMScheduler

# Custom modules
from models import UNETLatentEdgePredictor, SketchSimplificationNetwork
from pipeline import SketchGuidedText2Image


c:\Users\zaccu\OneDrive\Documents\GitHub\U-SketchPlus\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\zaccu\OneDrive\Documents\GitHub\U-SketchPlus\.venv\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# Configure and load sketch simplification network 

sketch_simplifier = SketchSimplificationNetwork().to(device)
sketch_simplifier.load_state_dict(torch.load("models-checkpoints/model_gan.pth"))

sketch_simplifier.eval()
sketch_simplifier.requires_grad_(False)

C:\Users\zaccu\AppData\Local\Temp\ipykernel_23432\1103916148.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sketch_simplifier.load_state_dict(torch.load("models-checkpo

SketchSimplificationNetwork(
  (0): Conv2d(1, 48, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (1): ReLU()
  (2): Conv2d(48, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU()
  (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (7): ReLU()
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (13): ReLU()
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU()
  (16): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU()
  (18): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (19): ReLU()
  (20): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [4]:
# Load Stable Diffusion Pipeline
stable_diffusion_1_5 = "benjamin-paine/stable-diffusion-v1-5"

In [5]:
stable_diffusion=StableDiffusionPipeline.from_pretrained(
    stable_diffusion_1_5,
    torch_dtype=torch.float16,
    safety_checker=None  # Skip the safety checker if it's not required
)
vae = stable_diffusion.vae.to(device)
unet = stable_diffusion.unet.to(device)
tokenizer = stable_diffusion.tokenizer
text_encoder = stable_diffusion.text_encoder.to(device) 

vae.eval()
unet.eval()
text_encoder.eval()

text_encoder.requires_grad_(False)
unet.requires_grad_(False)

Loading pipeline components...:  33%|███▎      | 2/6 [00:00<00:01,  2.77it/s]c:\Users\zaccu\OneDrive\Documents\GitHub\U-SketchPlus\.venv\lib\site-packages\transformers\models\clip\feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 6/6 [00:07<00:00,  1.19s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing it

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

In [6]:
# Load U-Net latent edge predictor
checkpoint = torch.load("models-checkpoints/unet_latent_edge_predictor_checkpoint.pt",map_location=torch.device('cpu'))

LEP_UNET = UNETLatentEdgePredictor(9320, 4, 9).to(device)
LEP_UNET.load_state_dict(checkpoint["model_state_dict"])

LEP_UNET.eval()
LEP_UNET.requires_grad_(False)

C:\Users\zaccu\AppData\Local\Temp\ipykernel_23432\323835721.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("models-checkpoints/unet_latent_edge_

UNETLatentEdgePredictor(
  (e1): encoder_block(
    (conv): convolutional_block(
      (conv1): Conv2d(9320, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (e2): encoder_block(
    (conv): convolutional_block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (

In [7]:
import numpy 
# Set Scheduler
noise_scheduler = DDIMScheduler(
        beta_start = 0.00085,
        beta_end = 0.012,
        beta_schedule = "scaled_linear",
        num_train_timesteps = 1000,
        clip_sample = False,
    )

In [8]:
import numpy
print(numpy.__version__)

1.26.4


In [12]:
# Initialize Text-guided Text-to-Image synthesis pipeline
import tqdm

pipeline = SketchGuidedText2Image(stable_diffusion_pipeline = stable_diffusion, 
                                  unet = unet, vae = vae, 
                                  text_encoder = text_encoder, 
                                  lep_unet = LEP_UNET, scheduler = noise_scheduler, 
                                  tokenizer = tokenizer,
                                  sketch_simplifier = sketch_simplifier,
                                  device = device)

In [19]:
import itertools
import matplotlib.pyplot as plt
from PIL import Image

def tune_hyperparameters(pipeline, prompt, edge_maps, seed=None):
    """
    Performs a grid search over hyperparameters for the inference process.
    
    Parameters:
      pipeline: an instance of your SketchGuidedText2Image pipeline.
      prompt: list or string prompt.
      edge_maps: list of PIL Images for edge maps.
      seed: optional seed for reproducibility.
    
    Returns:
      results: a list of dictionaries, each containing the hyperparameters and the output images.
    """
    # Define the grid of hyperparameters to explore
    num_inference_timesteps_list = [12,15,20]  # using 12 steps for training constraints
    classifier_guidance_strength_list = [4, 6, 8]  # try a few values lower than the demo’s 8
    sketch_guidance_strength_list = [0.5, 1.0, 0.7, 1.6]  # scaled down values relative to the original demo
    guidance_steps_perc_list = [0.5]  # keeping relative percentage the same

    results = []
    
    # Loop over all combinations
    for (num_steps, cls_str, skt_str, gd_perc) in itertools.product(
            num_inference_timesteps_list,
            classifier_guidance_strength_list,
            sketch_guidance_strength_list,
            guidance_steps_perc_list):
        
        print(f"Testing: num_steps={num_steps}, classifier={cls_str}, sketch={skt_str}, guidance%={gd_perc}")
        
        # Call the Inference function from your pipeline
        output = pipeline.Inference(
            prompt=prompt,
            num_images_per_prompt=1,
            edge_maps=edge_maps,
            negative_prompt=None,
            num_inference_timesteps=num_steps,
            classifier_guidance_strength=cls_str,
            sketch_guidance_strength=skt_str,
            seed=seed,
            simplify_edge_maps=False,
            guidance_steps_perc=gd_perc,
        )
        
        results.append({
            "num_steps": num_steps,
            "classifier_guidance_strength": cls_str,
            "sketch_guidance_strength": skt_str,
            "guidance_steps_perc": gd_perc,
            "result": output  # output contains keys like "generated_image" and "edge_map"
        })
    
    return results

In [20]:
from tqdm import tqdm

# Make sure your pipeline instance is already created and configured as in your demo
prompt = ["Lego car"]
edge_maps = [Image.open("Lego_256x256/sketch/Car-1.jpg/sketchs32strokes_Car-1.jpg")]
seed = 1000

inverse_diffusion = pipeline.Inference(
    prompt=["Lego car "],
    num_images_per_prompt=1,
    edge_maps=edge_maps,
    negative_prompt=None,
    num_inference_timesteps=50,
    classifier_guidance_strength=8,
    sketch_guidance_strength=1.6,
    seed=seed,
    simplify_edge_maps=False,
    guidance_steps_perc=0.5,
)

# Run the hyperparameter tuner
tuning_results = tune_hyperparameters(pipeline, prompt, edge_maps, seed=seed)

# Visualize the outputs
n_results = len(tuning_results)
fig, axs = plt.subplots(1, n_results, figsize=(4 * n_results, 4))
if n_results == 1:
    axs = [axs]
for ax, res in zip(axs, tuning_results):
    # Assume the generated image is under the key "generated_image" and is a list of PIL images
    generated_img = res["result"]["generated_image"][0]
    ax.imshow(generated_img)
    ax.axis("off")
    title = (f"Steps: {res['num_steps']}\n"
             f"Cls: {res['classifier_guidance_strength']}\n"
             f"Sketch: {res['sketch_guidance_strength']}\n"
             f"Perc: {res['guidance_steps_perc']}")
    ax.set_title(title)
plt.show()

AttributeError: type object 'tqdm_asyncio' has no attribute 'tqdm'